In [3]:
# find MovieLense 20M dataset on kaggle
# download rating.csv

In [11]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.utils import shuffle

from keras.models import Model
from keras.layers import Input, Embedding, Dot, Add, Flatten, Dense, Concatenate
from keras.layers import Dropout, BatchNormalization, Activation
from keras.regularizers import l2
from keras.optimizers import SGD, Adam

Using TensorFlow backend.


<h2>Preprocessing<h2>

In [ ]:
df = pd.read_csv('rating.csv')

In [ ]:
df.head(3)

,userId,movieId,rating,timestamp
0,1,2,3.5,2005-04-02 23:53:47
1,1,29,3.5,2005-04-02 23:31:16
2,1,32,3.5,2005-04-02 23:33:39


In [ ]:
# IDs from 1 , we need to get them to start from 0
# thankfully no missing number in between
df.userId -= 1 

In [ ]:
# movie ids from 1 with missing number in between
# make user id go from 0 - N-1 ,
# not all movies have ids
# this takes some time
movie_ids = set(df.movieId.values)
movie2idx = {k:v for v,k in enumerate(movie_ids)}
df['movie_idx'] = df.apply(lambda row: movie2idx[row.movieId] , axis = 1)


In [ ]:
# drop unecessary timestamp col
df = df.drop(columns=['timestamp'])

In [ ]:
# since that took some time lets save
df.to_csv('rating_1.csv')

In [12]:
df = pd.read_csv('rating_1.csv')

In [13]:
N = df.userId.max() + 1 # num of user
M = df.movie_idx.max() + 1 # num of movies

In [38]:
df = shuffle(df)
cutoff = int(0.8*len(df))
df_train = df.iloc[:cutoff]
df_test = df.iloc[cutoff:]

In [15]:
K = 10 # latent dimensionality
mu = df_train.rating.mean() 
epochs = 15
reg = 0. # regularisation penalty

In [27]:
u = Input(shape=(1,))
m = Input(shape=(1,))
u_embedding = Embedding(N,K)(u) # (N,1,K)
m_embedding = Embedding(M,K)(m) # (M,1,K)


In [28]:
# Matrix Factorisation - main brach
u_bias = Embedding(N,1)(u) # (N,1,1)
m_bias = Embedding(M,1)(m) # (N,1,1)
x = Dot(axes=2)([u_embedding,m_embedding]) # (N,1,1)
x = Add()([x,u_bias,m_bias])
x = Flatten()(x) # (N,1)

In [29]:
#side branch ANN , since MF is a linear model hopefully ANN captures more infor
u_embedding = Flatten()(u_embedding) # (N,K)
m_embedding = Flatten()(m_embedding) # (N,K)
y = Concatenate()([u_embedding,m_embedding]) # (N,2K)
y = Dense(400)(y)
y = Activation('elu')(y)
y = Dense(1)(y)

In [31]:
# merge
x = Add()([x,y])

In [41]:
model = Model(inputs=[u,m],outputs=x)
model.compile(loss='mse',optimizer=SGD(lr=0.08,momentum=0.9),metrics=['mse'])

In [42]:
r = model.fit(
    x = [df_train.userId.values,df_train.movie_idx.values],
    y = df_train.rating.values - mu,
    epochs = epochs,
    batch_size = 1024,
    validation_data=([df_test.userId.values,df_test.movie_idx.values],df_test.rating.values - mu)

)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 16000210 samples, validate on 4000053 samples
Epoch 1/15
16000210/16000210 [==============================] - 182s 11us/step - loss: 0.7269 - mse: 0.7269 - val_loss: 0.7240 - val_mse: 0.7240
Epoch 2/15
16000210/16000210 [==============================] - 185s 12us/step - loss: 0.6984 - mse: 0.6984 - val_loss: 0.7008 - val_mse: 0.7008
Epoch 3/15
16000210/16000210 [==============================] - 183s 11us/step - loss: 0.6879 - mse: 0.6879 - val_loss: 0.7002 - val_mse: 0.7002
Epoch 4/15
16000210/16000210 [==============================] - 183s 11us/step - loss: 0.6822 - mse: 0.6822 - val_loss: 0.6968 - val_mse: 0.6968
Epoch 5/15
16000210/16000210 [==============================] - 182s 11us/step - loss: 0.6778 - mse: 0.6778 - val_loss: 0.6902 - val_mse: 0.6902
Epoch 6/15
16000210/16000210 [==============================] - 184s 11us/step - loss: 0.6706 - mse: 0.6706 - val_loss: 0.6890 - val_mse: 0.6890
Epoch 7/15
16000210/16000210 [==============================] - 180s 11us/s